# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [173]:
import pandas as pd
import numpy as np

In [174]:
propostas = pd.read_csv('application_record.csv', na_values=('na', 'NaN', 'nan'))
pagamentos = pd.read_csv('pagamentos_largo.csv', na_values=('na', 'NaN', 'nan'))

### Tarefa 1

In [175]:
# 1) Seu código aqui

In [176]:
pagamentos.head(5)

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [177]:
# tamanho da dataset
pagamentos.shape

(20937, 26)

In [178]:
# cria novo conjunto de indices

indices = list(range(0,25))
indices.insert(0, 'ID')

pagamentos.set_axis(indices, axis='columns', inplace=True)

# verificando se a substituição deu certo
pagamentos.head()

/tmp/ipykernel_103033/4056414580.py:6: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  pagamentos.set_axis(indices, axis='columns', inplace=True)


,ID,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [179]:
# verificando se há 'missings'
pagamentos.isna().any()

ID    False
0     False
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
dtype: bool

Não será feito a exclusão das linhas que possuem dados vazios, pois representa uma quantidade de dados signitificativa do dataset.

In [180]:
# verificando os tipos de variáveis

pagamentos.dtypes

ID     int64
0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
23    object
24    object
dtype: object

A variável que indica o status de pagamento é do tipo objeto.

In [181]:
# define a função de classificação conforme definido no início

def classifica_indicadores(valor) -> str:

    if valor == '0':
        return '1-29'
    if valor == '1':
        return '30-59'
    if valor == '2':
        return '60-89'
    if valor == '3':
        return '90-119'
    if valor == '4':
        return '120-149'
    if valor == '5':
        return '150-'
    if valor == 'C':
        return 'paid off that month'
    if valor == 'X':
        return 'no loan for the month'
    else:
        return 'NaN'

In [182]:
# define a função de classificação conforme o default -> bom: 0 e mal: 1

def classifica_default(valor) -> str:

  if valor == '0':
      return 'bom'
  if valor == '1':
      return 'bom'
  if valor == '2':
      return 'mau'
  if valor == '3':
      return 'mau'
  if valor == '4':
      return 'mau'
  if valor == '5':
      return 'mau'
  if valor == 'C':
      return 'bom'
  if valor == 'X':
      return 'bom'
  else:
      return 'NaN'

In [183]:
# cria uma cópia da lista de pagamentos para manipulação

pg_indicadores = pagamentos.copy()

In [184]:
# usando a lista pg_indicadores, a classificação é feita criando um novo dataframe

for i in range(0,25):
    pg_indicadores[i] = pg_indicadores[i].map(classifica_indicadores)

In [185]:
# teste na coluna do primeiro mês

pg_indicadores[0].value_counts()

1-29                   20302
paid off that month      550
30-59                     78
120-149                    2
150-                       2
60-89                      2
90-119                     1
Name: 0, dtype: int64

In [186]:
# usando a lista pg_indicadores, a variável default por mês é avaliada

pg_default = pagamentos.copy()

for i in range(0,25):
    pg_default[i] = pg_default[i].map(classifica_default)

In [187]:
# teste na coluna do segundo mês

pg_default[1].value_counts()

bom    19198
NaN     1721
mau       18
Name: 1, dtype: int64

### Tarefa 2


In [188]:
# 2) Seu código aqui

In [189]:
pg_default.shape

(20937, 26)

In [190]:
pg_default.head()

,ID,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,5001718,bom,bom,bom,bom,bom,bom,bom,bom,NaN,...,NaN,bom,NaN,bom,bom,bom,bom,NaN,bom,NaN
1,5001719,bom,bom,bom,bom,bom,bom,bom,bom,bom,...,bom,bom,bom,bom,bom,bom,bom,bom,bom,bom
2,5001720,bom,bom,bom,bom,bom,bom,bom,bom,bom,...,bom,bom,bom,bom,bom,bom,bom,bom,bom,bom
3,5001723,bom,bom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,bom,bom,bom,bom,bom,NaN,NaN
4,5001726,bom,bom,bom,bom,bom,bom,bom,bom,bom,...,bom,bom,bom,bom,bom,bom,bom,bom,bom,bom


In [191]:
# função para verificar se 'mau' está na lista de colunas de cada cliente ('ID')

def contador_list(lista: list) -> int:
    
    # inicia o contador
    count = 0
    # deleta o primeiro elemento, que é o 'ID'
    del lista[0]
    
    if 'mau' in lista:
        count = 1
    else:
        count = 0
    
    return int(count)

Se o cliente tiver sido mau pagador em algum mês, a flag True será exibida em Status. Se ele for um bom pagador, a flag False é exibida.

In [192]:
# cria uma lista vazia e a coluna Status, também vazia, no pg_default

lista = []
pg_default['Status'] = ''

for index in range(0,len(pg_default)):
    # converte os elementos de cada linha em uma lista
    lista = pg_default.iloc[index].tolist()
    #aplica a função contador_list
    verificador = contador_list(lista)
    
    if verificador == 1:
        pg_default['Status'].iloc[index] = True
    if verificador == 0:
        pg_default['Status'].iloc[index] = False

/tmp/ipykernel_103033/1369023949.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pg_default['Status'].iloc[index] = False
/tmp/ipykernel_103033/1369023949.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pg_default['Status'].iloc[index] = True


In [193]:
# verificando a coluna Status e seus valores

print(f"{pg_default['Status']} \n")

print(pg_default['Status'].value_counts())

0        False
1        False
2        False
3        False
4        False
         ...  
20932    False
20933    False
20934    False
20935    False
20936    False
Name: Status, Length: 20937, dtype: object 

False    20506
True       431
Name: Status, dtype: int64


In [215]:
# calcula a porcentagem de maus pagadores

bons_pg = pg_default[pg_default['Status'] == False]
maus_pg = pg_default[pg_default['Status'] == True]

print(f'{round(len(maus_pg)/len(pg_default)*100,2)}% de maus pagadores, em pelo menos um mês.')


2.06% de maus pagadores, em pelo menos um mês.


### Tarefa 3 e 4

In [195]:
# Juntar a base de clientes na df pagamentos com a base de clientes na df propostas
# Usar o ID do cliente como chave principal

propostas.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [205]:
# verificando o tamanho de cada dataset

print(f'clientes com cartão: {len(pg_default)}')
print(f'clientes que pediram análise de propostas (não necessariamente possuem cartão): {len(propostas)}')

clientes com cartão: 20937
clientes que pediram análise de propostas (não necessariamente possuem cartão): 438557


In [202]:
# cria um df associando apenas o ID e o Status

pg_status = pg_default[['ID','Status']]
pg_status

,ID,Status
0,5001718,False
1,5001719,False
2,5001720,False
3,5001723,False
4,5001726,False
...,...,...
20932,5150475,False
20933,5150476,False
20934,5150480,False
20935,5150482,False


In [210]:
# cria um df que é a junção de 'pg_status' e 'propostas', mas apenas com os clientes que possuem cartão de crédito
# e fizeram propostas

merged_propostas = pd.merge(pg_status, propostas, on = "ID", how = "inner")
merged_propostas.head()

,ID,Status,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008806,False,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
1,5008810,False,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
2,5008811,False,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
3,5008825,False,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0
4,5008826,False,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0


### Tarefa 5

In [211]:
# checar o número de defaults no novo banco cruzado

merged_propostas['Status'].value_counts()

False    16260
True       390
Name: Status, dtype: int64

In [217]:
merged_bons = merged_propostas[merged_propostas['Status'] == False]
merged_maus = merged_propostas[merged_propostas['Status']]

print(f'{round(len(merged_maus)/len(merged_propostas)*100,2)}% de maus pagadores, em pelo menos um mês.')

2.34% de maus pagadores, em pelo menos um mês.


A porcentagem de maus pagadores se manteve proporcinal no dataset cruzado, cerca de 2.34% dos clientes que possuem cartão de crédito e fizeram uma proposta.